# Imports

In [ ]:
!pip install ultralytics -q
!pip install roboflow -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.5 MB/s eta 0:00:00


# Download dataset

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="4WOW3qLWIEesUVhweOCi")
project = rf.workspace("innopolis-university-5kqay").project("grocery-products-detection")
dataset = project.version(5).download("yolov8")


print(f"Location: {dataset.location}")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.206, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Grocery-products-detection-5 in yolov8:: 100%|██████████| 1170/1170 [00:00<00:00, 2653.96it/s]


Location: /content/Grocery-products-detection-5


## Move data.yaml to the root and go into the dataset folder

In [ ]:
!cp Grocery-products-detection-5/data.yaml data.yaml
%cd Grocery-products-detection-5

/content/Grocery-products-detection-5


# Train YOLOv8 small

In [ ]:
# AutoBatch + cache for data loading + patience 50 epochs
!yolo detect train \
project=InnoGroceryDetection \
model=yolov8s.pt \
data=../data.yaml \
pretrained=yolov8s.pt \
batch=-1 \
cache=True \
verbose=True \
epochs=300 \
patience=50 \
imgsz=640

Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.0.206 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=../data.yaml, epochs=300, patience=50, batch=-1, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=InnoGroceryDetection, name=train, exist_ok=False, pretrained=yolov8s.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mask

# Training dataset exploration

# Validate the trained model

In [ ]:
!yolo detect val \
model=InnoGroceryDetection/train/weights/best.pt \
save_json=True \
conf= 0.715 \
iou= 0.5 \
data=../data.yaml

Ultralytics YOLOv8.0.206 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11129841 parameters, 0 gradients, 28.5 GFLOPs
val: Scanning /content/Grocery-products-detection-5/valid/labels.cache... 48 images, 0 backgrounds, 0 corrupt: 100% 48/48 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/Grocery-products-detection-5/valid/images/Bread4_jpg.rf.1986c951a4ed26bab71d6f42c8fbfab7.jpg: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 3/3 [00:04<00:00,  1.35s/it]
                   all         48        271      0.946      0.855      0.912      0.834
                Apples         48         26       0.92      0.885      0.934      0.862
               Bananas         48         35          1      0.771      0.886      0.808
                 Bread         48         16      0.875      0.875      0.917       0.83
                Carrot         48         36       0.81      0.472

# Deploy the model to roboflow project

In [ ]:
api_key=''
project = Roboflow(api_key=api_key).workspace("innopolis-university-5kqay").project("grocery-products-detection")
project.version(5).deploy(model_type='yolov8s', model_path=f'InnoGroceryDetection/train')


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.206, to fix: `pip install ultralytics==8.0.196`
Would you like to continue with the wrong version of ultralytics? y/n: y
View the status of your deployment at: https://app.roboflow.com/innopolis-university-5kqay/grocery-products-detection/5
Share your model with the world at: https://universe.roboflow.com/innopolis-university-5kqay/grocery-products-detection/model/5
